# <font color='#42AAFF'>Содержание</font><a id='toc0_'></a>  <a id='toc0_'></a>    
1. [<font color='#42AAFF'>Импорт предобработанных данных</font>](#toc1_)    
2. [<font color='#42AAFF'>Проверка признаков на мультиколлинеарность</font>](#toc2_)    
3. [<font color='#42AAFF'>Оптимизация FAISS</font>](#toc3_)    
3.1. [<font color='#42AAFF'>Обучение индексов при разном количестве кластеров</font>](#toc3_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import faiss
import time
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 1. <a id='toc1_'></a>[<font color='#42AAFF'>Импорт предобработанных данных</font>](#toc0_)

In [2]:
features_base = pd.read_csv('features_base.csv',index_col=0)
features_train = pd.read_csv('features_train.csv',index_col=0)
features_test = pd.read_csv('features_test.csv',index_col=0)
target_train = pd.read_csv('target_train.csv',index_col=0)
target_test = pd.read_csv('target_test.csv',index_col=0)

Колонки с признаками, имеющими явно не стандартные распределения:

In [3]:
cols1 = ['6', '21', '25', '33', '44', '59', '65', '70']

# 2. <a id='toc2_'></a>[<font color='#42AAFF'>Проверка признаков на мультиколлинеарность</font>](#toc0_)

Проверим исходную базу на коллинеарность признаков с помощью Variance Inflation Factor:

In [4]:
def variance_inflation_factors(x):
    model = LinearRegression()
    vif = pd.DataFrame(columns=['features','vif'])
    for i in range(x.shape[1]):        
        x_i = x.drop(x.columns[i], axis=1)
        y_i = x[x.columns[i]]
        model_i = model.fit(x_i, y_i)
        y_i_pred = model_i.predict(x_i)        
        r2_i = r2_score(y_i, y_i_pred)        
        vif_i = 1 / (1 - r2_i)
        vif.loc[i,'features'] = x.columns[i]
        vif.loc[i,'vif'] = vif_i
        vif = vif.sort_values(by='vif',ascending=False).reset_index(drop=True)
    return vif

Если VIF>5 то будем считать признак коллинеарным, определим VIF:

In [5]:
variance_inflation_factors(features_base)

,features,vif
0,63,1.229038
1,35,1.147099
2,57,1.102206
3,10,1.093029
4,45,1.0893
...,...,...
67,6,1.019902
68,18,1.019497
69,12,1.017122
70,65,1.00188


Мультиколлинеарных признаков не обнаружили.

# 3. <a id='toc3_'></a>[<font color='#42AAFF'>Оптимизация FAISS</font>](#toc0_)

## 3.1. <a id='toc3_1_'></a>[<font color='#42AAFF'>Обучение индексов при разном количестве кластеров</font>](#toc0_)

In [ ]:
num, dim = features_base.shape
quantiser = faiss.IndexFlatL2(dim) 
n_clusters_list = [15, 30, 50, 100, 250, 500, 1000, 2000, 3000]

Обучим индексы при разном количестве кластеров и сохраним результаты:

In [ ]:
%%time
for j,n_clusters in enumerate(n_clusters_list):
    index = faiss.IndexIVFFlat(quantiser, dim, n_clusters)
    index.train(np.ascontiguousarray(features_base.values).astype('float32'))
    index.add(np.ascontiguousarray(features_base.values).astype('float32'))
    faiss.write_index(index, 'index_'+ str(n_clusters))

Wall time: 40min 18s


## 4. <a id='toc3_'></a>[<font color='#42AAFF'>Влияние доли кластеров поиска</font>](#toc0_)

In [4]:
def faiss_accuracy(features_base,base_index,features_train,target_train,n_clusters_search,n_samples,index):    
    start_time = time.time()    
    index.nprobe = n_clusters_search
    vecs, idx = index.search(np.ascontiguousarray(features_train.values).astype('float32'), n_samples)
    t_search = time.time() - start_time
    acc = 0
    for target, el in zip(target_train[target_train.columns[0]].values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el])
    acc = 100 * acc / len(idx) 
    return acc, t_search  

In [5]:
results = pd.DataFrame(columns=['n_clusters','n_clusters_search','n_clusters_ratio','n_samples','time_search','accuracy'])
n_clusters_list = [15, 100, 250, 500, 1000, 2000, 3000]
n_clusters_ratio = [0.1, 0.2, 0.5]
n_samples_list = [5, 200, 500]

ind=0;
for n_clusters in n_clusters_list:
    index = faiss.read_index('index_'+ str(n_clusters))
    base_index = {k: v for k, v in enumerate(features_base.index.to_list())}
    for ratio in n_clusters_ratio:
        n_clusters_search = int(np.ceil(ratio*n_clusters))
        for n_samples in n_samples_list:
            acc, t_search  = faiss_accuracy(
                features_base, base_index,features_train,target_train,n_clusters_search,n_samples,index)            
            results.loc[ind,'n_clusters'] = n_clusters
            results.loc[ind,'n_clusters_search'] = n_clusters_search
            results.loc[ind,'n_clusters_ratio'] = ratio
            results.loc[ind,'n_samples'] = n_samples            
            results.loc[ind,'accuracy'] = acc
            results.loc[ind,'time_search'] = t_search  
            display(results[ind:ind+1])    
            ind += 1 

,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
0,15,2,0.1,5,425.775919,54.554


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
1,15,2,0.1,200,429.399675,65.486


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
2,15,2,0.1,500,424.060921,67.148


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
3,15,3,0.2,5,630.815978,58.191


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
4,15,3,0.2,200,631.957546,70.751


...

,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
47,2000,200,0.1,500,274.320039,82.231


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
48,2000,400,0.2,5,547.640897,64.0


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
49,2000,400,0.2,200,547.907029,80.196


,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
50,2000,400,0.2,500,548.360596,82.735


KeyboardInterrupt: 

In [6]:
results.to_csv('results')

In [7]:
results.head(5)

,n_clusters,n_clusters_search,n_clusters_ratio,n_samples,time_search,accuracy
0,15,2,0.1,5,425.775919,54.554
1,15,2,0.1,200,429.399675,65.486
2,15,2,0.1,500,424.060921,67.148
3,15,3,0.2,5,630.815978,58.191
4,15,3,0.2,200,631.957546,70.751
